













no_repeat_ngram_size=2: تمنع تكرار الجمل المكونة من كلمتين متتاليتين، وهي أيضًا السلوك الافتراضي.
بإزالة هذه الخيارات غير الضرورية، يصبح الكود أكثر إيجازًا وسهولة في القراءة




accelerate launch train_text_to_image.py   --pretrained_model_name_or_path=$MODEL_NAME   --dataset_name=$dataset_name   --use_ema   --resolution=512 --center_crop --random_flip   --train_batch_size=1   --gradient_accumulation_steps=8   --mixed_precision="fp16"   --max_train_steps=15000   --learning_rate=1e-05   --max_grad_norm=1   --lr_scheduler="constant" --lr_warmup_steps=0   --output_dir="sd-pokemon-model"  --validation_prompt=Yoda --num_validation_images=8  --validation_steps=1000  --enable_xformers_memory_efficient_attention












https://pytorch.org/tutorials/intermediate/torch_compile_tutorial.html
https://huggingface.co/docs/diffusers/main/optimization/memory
https://huggingface.co/docs/diffusers/main/optimization/fp16
https://huggingface.co/docs/diffusers/main/optimization/torch2.0

torch.nn.functional.scaled_dot_product_attention (SDPA) هو اهتمام محسن وفعال للذاكرة (على غرار xFormers) يتيح تلقائيا العديد من التحسينات الأخرى اعتمادا على مدخلات النموذج ونوع وحدة معالجة الرسومات. يتم تمكين SDPA افتراضيا إذا كنت تستخدم PyTorch 2.0 وأحدث إصدار من 🤗 Diffusers ، لذلك لا تحتاج إلى إضافة أي شيء إلى التعليمات البرمجية الخاصة بك.

ومع ذلك ، إذا كنت تريد تمكينه بشكل صريح ، فيمكنك تعيين DiffusionPipeline لاستخدامه AttnProcessor2_0:
  import torch
  from diffusers import DiffusionPipeline
+ from diffusers.models.attention_processor import AttnProcessor2_0

  pipe = DiffusionPipeline.from_pretrained("stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16, use_safetensors=True).to("cuda")
+ pipe.unet.set_attn_processor(AttnProcessor2_0())

  prompt = "a photo of an astronaut riding a horse on mars"
  image = pipe(prompt).images[0]
ززززززززززززززززززززززززززز
  import torch
  from diffusers import DiffusionPipeline

  pipe = DiffusionPipeline.from_pretrained("stable-diffusion-v1-5/stable-diffusion-v1-5", torch_dtype=torch.float16, use_safetensors=True).to("cuda")
+ pipe.unet.set_default_attn_processor()

  prompt = "a photo of an astronaut riding a horse on mars"
  image = pipe(prompt).images[0]
ززززززززززززززززززززززززززززززز
from diffusers import DiffusionPipeline
import torch

path = "stable-diffusion-v1-5/stable-diffusion-v1-5"

run_compile = True  # Set True / False

pipe = DiffusionPipeline.from_pretrained(path, torch_dtype=torch.float16, use_safetensors=True)
pipe = pipe.to("cuda")
pipe.unet.to(memory_format=torch.channels_last)

if run_compile:
    print("Run torch compile")
    pipe.unet = torch.compile(pipe.unet, mode="reduce-overhead", fullgraph=True)

prompt = "ghibli style, a fantasy landscape with castles"

for _ in range(3):
    images = pipe(prompt=prompt).images
ززززززززززززززززززززززززززززززززز

from diffusers import StableDiffusionImg2ImgPipeline
from diffusers.utils import load_image
import torch

url = "https://raw.githubusercontent.com/CompVis/stable-diffusion/main/assets/stable-samples/img2img/sketch-mountains-input.jpg"

init_image = load_image(url)
init_image = init_image.resize((512, 512))

path = "stable-diffusion-v1-5/stable-diffusion-v1-5"

run_compile = True  # Set True / False

pipe = StableDiffusionImg2ImgPipeline.from_pretrained(path, torch_dtype=torch.float16, use_safetensors=True)
pipe = pipe.to("cuda")
pipe.unet.to(memory_format=torch.channels_last)

if run_compile:
    print("Run torch compile")
    pipe.unet = torch.compile(pipe.unet, mode="reduce-overhead", fullgraph=True)

prompt = "ghibli style, a fantasy landscape with castles"

for _ in range(3):
    image = pipe(prompt=prompt, image=init_image).images[0]
زززززززززززززززززززززززززززززززز
from diffusers import StableDiffusionInpaintPipeline
from diffusers.utils import load_image
import torch

img_url = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo.png"
mask_url = "https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo_mask.png"

init_image = load_image(img_url).resize((512, 512))
mask_image = load_image(mask_url).resize((512, 512))

path = "runwayml/stable-diffusion-inpainting"

run_compile = True  # Set True / False

pipe = StableDiffusionInpaintPipeline.from_pretrained(path, torch_dtype=torch.float16, use_safetensors=True)
pipe = pipe.to("cuda")
pipe.unet.to(memory_format=torch.channels_last)

if run_compile:
    print("Run torch compile")
    pipe.unet = torch.compile(pipe.unet, mode="reduce-overhead", fullgraph=True)

prompt = "ghibli style, a fantasy landscape with castles"

for _ in range(3):
    image = pipe(prompt=prompt, image=init_image, mask_image=mask_image).images[0]
زززززززززززززززززززززززززززززززززززززززززززز

ثم اتصل ب enable_xformers_memory_efficient_attention() على خط الأنابيب:

نسخ
from diffusers import DiffusionPipeline
import torch

pipe = DiffusionPipeline.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
    use_safetensors=True,
).to("cuda")

pipe.enable_xformers_memory_efficient_attention()

with torch.inference_mode():
    sample = pipe("a small cat")

# optional: You can disable it via
# pipe.disable_xformers_memory_efficient_attention()
زززززززززززززززززززززززززززززززززززززززززززززززززززززز

import torch
from diffusers import CogVideoXPipeline, CogVideoXTransformer3DModel
from diffusers.utils import export_to_video

model_id = "THUDM/CogVideoX-5b"

# Load the model in bfloat16 and enable layerwise casting
transformer = CogVideoXTransformer3DModel.from_pretrained(model_id, subfolder="transformer", torch_dtype=torch.bfloat16)
transformer.enable_layerwise_casting(storage_dtype=torch.float8_e4m3fn, compute_dtype=torch.bfloat16)

# Load the pipeline
pipe = CogVideoXPipeline.from_pretrained(model_id, transformer=transformer, torch_dtype=torch.bfloat16)
pipe.to("cuda")

prompt = (
    "A panda, dressed in a small, red jacket and a tiny hat, sits on a wooden stool in a serene bamboo forest. "
    "The panda's fluffy paws strum a miniature acoustic guitar, producing soft, melodic tunes. Nearby, a few other "
    "pandas gather, watching curiously and some clapping in rhythm. Sunlight filters through the tall bamboo, "
    "casting a gentle glow on the scene. The panda's face is expressive, showing concentration and joy as it plays. "
    "The background includes a small, flowing stream and vibrant green foliage, enhancing the peaceful and magical "
    "atmosphere of this unique musical performance."
)
video = pipe(prompt=prompt, guidance_scale=6, num_inference_steps=50).frames[0]
export_to_video(video, "output.mp4", fps=8)
ززززززززززززززززززززززززززززززززززززززززززززززززززززززز


يُفضل دائمًا استخدام if hasattr لضمان توافق الكود مع نماذج وخطوط أنابيب مختلفة وتجنب الأخطاء.

هذا يجعل الكود الخاص بك أكثر قابلية للصيانة وقابلية للتكيف مع التغييرات في مكتبات Transformers و xformers.
!pip install torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0
شغال
import torch
from transformers import pipeline
import xformers  # Import xformers

# Enable xformers memory efficient attention
pipe = pipeline(model="openai-community/gpt2", torch_dtype=torch.bfloat16, device_map="auto")

# Check if xformers is available and enable it
if hasattr(pipe.model, 'enable_xformers_memory_efficient_attention'):
    pipe.model.enable_xformers_memory_efficient_attention()
else:
    print("xformers is not available for this model.")

output = pipe("This is a cool example!", do_sample=True, top_p=0.95)
print(output)
..ززززززززززززززززززززززززززززززززززززززززززززززززززز


شرح الأوامر باللغة العربية:
1. torch.set_grad_enabled(False):

هذا الأمر يقوم بتعطيل حساب التدرجات (gradients) في PyTorch.

التدرجات هي قيم تُستخدم في تدريب النماذج العصبية لتحديث معلمات النموذج وتحسين أدائه. عند القيام بـ الاستدلال (inference)، أي استخدام النموذج المُدرّب بالفعل لتوليد مخرجات جديدة، لا نحتاج إلى حساب التدرجات.

لذلك، يتم استخدام torch.set_grad_enabled(False) لتسريع عملية الاستدلال وتوفير مساحة الذاكرة، حيث لا يتم تخزين التدرجات.

2. torch.backends.cudnn.benchmark = True:

هذا الأمر يقوم بتفعيل خيار التحسين التلقائي (auto-tuner) في مكتبة cuDNN، وهي مكتبة تُستخدم لتسريع عمليات التعلم العميق على أجهزة GPU من NVIDIA.

التحسين التلقائي يعني أن cuDNN ستبحث عن أفضل خوارزمية لتنفيذ العمليات على GPU الخاص بك، مما يؤدي إلى تحسين الأداء وتسريع عملية الاستدلال.

لكن، هذا التحسين التلقائي قد يستغرق بعض الوقت في البداية، لذلك يُفضّل استخدامه فقط عند تشغيل النموذج عدة مرات بنفس حجم المدخلات.

بإختصار:

torch.set_grad_enabled(False): لتعطيل حساب التدرجات وتسريع الاستدلال.
torch.backends.cudnn.benchmark = True : لتحسين أداء cuDNN على GPU وتسريع الاستدلال.
أتمنى أن يكون هذا الشرح واضحًا ومفيدًا!










........................................



pipe.enable_model_cpu_offload()
pipe.enable_sequential_cpu_offload()
pipe.enable_vae_slicing()
pipe.enable_vae_tiling()
pipe.enable_xformers_memory_efficient_attention()
StableDiffusionPipeline.enable_model_cpu_offload
transformer = CogVideoXTransformer3DModel.from_pretrained(model_id, subfolder="transformer", torch_dtype=torch.bfloat16)
transformer.enable_layerwise_casting(storage_dtype=torch.float8_e4m3fn, compute_dtype=torch.bfloat16)

torch.backends.xformers.enabled = True
torch.set_grad_enabled(False)
torch.backends.cudnn.benchmark = True











































In [ ]:
!pip install ninja

In [1]:
# Same requirements as for the stable version above
!pip install --pre -U xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!wget https://huggingface.co/kilo40/xformers_whl/resolve/main/xformers-0.0.16%2B217b111.d20230113-cp310-cp310-linux_x86_64.whl

In [ ]:
!wget https://huggingface.co/multimodalart/xformers/resolve/main/xformers-0.0.14.dev0-cp38-cp38-linux_x86_64_t4.whl

In [ ]:
!wget https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.17/xformers-0.0.17+b6be33a.d20230315-cp39-cp39-linux_x86_64.whl

In [ ]:
!pip install https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.17/xformers-0.0.17+b6be33a.d20230315-cp39-cp39-linux_x86_64.whl

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from xformers.ops import enable_xformers_memory_efficient_attention
enable_xformers_memory_efficient_attention()
# تحميل المودل والتوكنайزر
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"  # قم بتعديله إلى اسم نموذجك الفعلي
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

def chat_with_llama(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=50, num_return_sequences=1, no_repeat_ngram_size=2, temperature=0.7)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

if __name__ == "__main__":
    print("مرحبا بك في شاشة الشات مع نموذج LLaMA!")
    while True:
        user_input = input("أنت: ")
        if user_input.lower() in ["خروج", "exit", "quit"]:
            print("شكراً! حاول مرة أخرى قريبًا.")
            break
        response = chat_with_llama(user_input)
        print(f"LLaMA: {response}\n")


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from xformers.ops import enable_xformers_memory_efficient_attention

# تمكين xformers للحصول على اهتمام ذاكرة أكثر كفاءة
enable_xformers_memory_efficient_attention()

# تحميل المودل والتوكنايزر
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"  # قم بتعديله إلى اسم نموذجك الفعلي
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,  # استخدام bfloat16 لتقليل استهلاك الذاكرة مع الحفاظ على الأداء
    device_map="auto"  # توزيع النموذج تلقائيًا على أجهزة GPU المتاحة
)

# تحديد الجهاز (GPU أو CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # نقل النموذج إلى الجهاز المناسب

# تمكين kv_cache لتحسين الأداء أثناء الاستدلال
model.config.use_cache = True

# مثال على كيفية إنشاء دخل واستخدام النموذج
input_text = "مرحباً بالعالم!"  # النص الذي تريد إكماله
inputs = tokenizer(input_text, return_tensors="pt").to(device)

# توليد النص باستخدام النموذج
with torch.no_grad():  # تعطيل الحسابات التفاضلية لأننا في مرحلة الاستدلال
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,  # عدد الرموز الجديدة التي سيتم توليدها
        do_sample=True,     # تمكين العينة العشوائية لإنتاج نتائج أكثر تنوعًا
        temperature=0.7,    # تحكم في درجة التنوع (قيم أقل تعني نتائج أكثر تحفظًا)
        pad_token_id=tokenizer.eos_token_id,  # تحديد رمز الإنهاء
        use_cache=True      # استخدام الكاش (kv_cache) لتحسين الأداء
    )

# فك التوكنات وإظهار النص المولد
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

https://github.com/huggingface/text-generation-inference/blob/main/server/text_generation_server/models/pali_gemma.py

In [ ]:
from io import BytesIO
from PIL import Image
import torch
import torch.distributed
from opentelemetry import trace
from typing import Iterable
from text_generation_server.models.vlm_causal_lm import (
    VlmCausalLMBatch,
    image_text_replacement,
)

from text_generation_server.pb.generate_pb2 import Request

tracer = trace.get_tracer(__name__)


class PaliGemmaBatch(VlmCausalLMBatch):
    @classmethod
    def batch_tokenized_inputs(
        cls, requests: Iterable[Request], tokenizer, processor, config
    ):
        batch_inputs = []
        image_inputs = []
        max_truncation = 0
        for r in requests:
            full_text = ""
            image_id = 0
            for chunk in r.input_chunks.chunks:
                chunk_type = chunk.WhichOneof("chunk")
                if chunk_type == "text":
                    full_text += "<bos>" + chunk.text + "\n"
                elif chunk_type == "image":
                    image = Image.open(BytesIO(chunk.image.data))
                    # TODO do_convert_RGB should be on by default ?
                    image = image.convert("RGB")
                    image_input = processor.image_processor(image, return_tensors="pt")
                    full_text += image_text_replacement(
                        processor, image_input, config, image_id
                    )
                    image_inputs.append(image_input)
                else:
                    raise RuntimeError(f"Invalid chunk type {chunk_type}")

            batch_inputs.append(full_text)
            max_truncation = max(max_truncation, r.truncate)

        batch_tokenized_inputs = tokenizer(
            batch_inputs,
            truncation=True,
            max_length=max_truncation,
            add_special_tokens=False,
        )["input_ids"]
        if image_inputs:
            image_input = image_inputs[0]
            new_image_inputs = {
                "pixel_values": torch.cat(
                    [img["pixel_values"] for img in image_inputs], dim=0
                ),
            }
            if "pixel_attention_mask" in image_input:
                new_image_inputs["pixel_attention_mask"] = torch.cat(
                    [img["pixel_attention_mask"] for img in image_inputs], dim=0
                )
            if "image_sizes" in image_input:
                new_image_inputs["image_sizes"] = torch.cat(
                    [img["image_sizes"] for img in image_inputs], dim=0
                )
            image_inputs = new_image_inputs
        else:
            image_inputs = None
        return batch_tokenized_inputs, image_inputs

In [ ]:
import torch
import numpy as np
from torch import nn
import torch.autograd.profiler as profiler

In [ ]:
!pip install transformers torch xformers


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# تفعيل خيار XFormers للذاكرة الفعالة لعمليات الانتباه
import torch
if torch.cuda.is_available():
    torch.backends.xformers.enabled = True
    print("XFormers is enabled.")

# تحميل النموذج والتوكنайزر
model_name = "gpt2"  # يمكنك استبداله بنموذجًا آخر مثل "gpt2-medium", "gpt2-large"، إلخ
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

def chat_with_gpt(prompt, max_length=150):
    # ترميز النص المدخل
    inputs = tokenizer(prompt, return_tensors="pt")
    # إنتاج الرد من النموذج
    outputs = model.generate(**inputs, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2, temperature=0.7)
    # فك ترميز الرد إلى نص سهل القراءة
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

if __name__ == "__main__":
    print("مرحبا بك في شاشة الشات مع نموذج GPT-2!")
    while True:
        user_input = input("أنت: ")
        if user_input.lower() in ["خروج", "exit", "quit"]:
            print("شكراً! حاول مرة أخرى قريبًا.")
            break
        response = chat_with_gpt(user_input)
        print(f"GPT-2: {response}\n")


In [ ]:
meta-llama/Llama-3.2-1B

deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from xformers.ops import enable_xformers_memory_efficient_attention

# تمكين xformers للحصول على اهتمام ذاكرة أكثر كفاءة
enable_xformers_memory_efficient_attention()

# تحميل المودل والتوكنايزر
model_name = "openai-community/gpt2"  # قم بتعديله إلى اسم نموذجك الفعلي
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,  # استخدام bfloat16 لتقليل استهلاك الذاكرة مع الحفاظ على الأداء
    device_map="auto"  # توزيع النموذج تلقائيًا على أجهزة GPU المتاحة
)

# تحديد الجهاز (GPU أو CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # نقل النموذج إلى الجهاز المناسب

# تمكين kv_cache لتحسين الأداء أثناء الاستدلال
model.config.use_cache = True

# مثال على كيفية إنشاء دخل واستخدام النموذج
input_text = "مرحباً بالعالم!"  # النص الذي تريد إكماله
inputs = tokenizer(input_text, return_tensors="pt").to(device)

# توليد النص باستخدام النموذج
with torch.no_grad():  # تعطيل الحسابات التفاضلية لأننا في مرحلة الاستدلال
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,  # عدد الرموز الجديدة التي سيتم توليدها
        do_sample=True,     # تمكين العينة العشوائية لإنتاج نتائج أكثر تنوعًا
        temperature=0.7,    # تحكم في درجة التنوع (قيم أقل تعني نتائج أكثر تحفظًا)
        pad_token_id=tokenizer.eos_token_id,  # تحديد رمز الإنهاء
        use_cache=True      # استخدام الكاش (kv_cache) لتحسين الأداء
    )

# فك التوكنات وإظهار النص المولد
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

ImportError: cannot import name 'enable_xformers_memory_efficient_attention' from 'xformers.ops' (/usr/local/lib/python3.11/dist-packages/xformers/ops/__init__.py)

In [3]:
!pip install --upgrade xformers

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
#from xformers.ops import enable_xformers_memory_efficient_attention # This line was replaced
from xformers.ops import memory_efficient_attention # Replacement with the correct function call

# تمكين xformers للحصول على اهتمام ذاكرة أكثر كفاءة
#enable_xformers_memory_efficient_attention() # This line was replaced
#memory_efficient_attention(True)
#torch.backends.xformers.enabled = True
enable_xformers_memory_efficient_attention()# This is the suggested replacement to enable memory efficient attention with xformers
# تحميل المودل والتوكنايزر
model_name = "openai-community/gpt2"  # قم بتعديله إلى اسم نموذجك الفعلي
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,  # استخدام bfloat16 لتقليل استهلاك الذاكرة مع الحفاظ على الأداء
    device_map="auto"  # توزيع النموذج تلقائيًا على أجهزة GPU المتاحة
)

# تحديد الجهاز (GPU أو CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # نقل النموذج إلى الجهاز المناسب

# تمكين kv_cache لتحسين الأداء أثناء الاستدلال
model.config.use_cache = True

# مثال على كيفية إنشاء دخل واستخدام النموذج
input_text = "مرحباً بالعالم!"  # النص الذي تريد إكماله
inputs = tokenizer(input_text, return_tensors="pt").to(device)

# توليد النص باستخدام النموذج
with torch.no_grad():  # تعطيل الحسابات التفاضلية لأننا في مرحلة الاستدلال
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,  # عدد الرموز الجديدة التي سيتم توليدها
        do_sample=True,     # تمكين العينة العشوائية لإنتاج نتائج أكثر تنوعًا
        temperature=0.7,    # تحكم في درجة التنوع (قيم أقل تعني نتائج أكثر تحفظًا)
        pad_token_id=tokenizer.eos_token_id,  # تحديد رمز الإنهاء
        use_cache=True      # استخدام الكاش (kv_cache) لتحسين الأداء
    )

# فك التوكنات وإظهار النص المولد
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

NameError: name 'enable_xformers_memory_efficient_attention' is not defined

In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# تفعيل خيار XFormers للذاكرة الفعالة لعمليات الانتباه
import torch
if torch.cuda.is_available():
    torch.backends.xformers.enabled = True
    print("XFormers is enabled.")

# تحميل النموذج والتوكنайزر
model_name = "gpt2"  # يمكنك استبداله بنموذجًا آخر مثل "gpt2-medium", "gpt2-large"، إلخ
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

def chat_with_gpt(prompt, max_length=150):
    # ترميز النص المدخل
    inputs = tokenizer(prompt, return_tensors="pt")
    # إنتاج الرد من النموذج
    outputs = model.generate(**inputs, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2, temperature=0.7)
    # فك ترميز الرد إلى نص سهل القراءة
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

if __name__ == "__main__":
    print("مرحبا بك في شاشة الشات مع نموذج GPT-2!")
    while True:
        user_input = input("أنت: ")
        if user_input.lower() in ["خروج", "exit", "quit"]:
            print("شكراً! حاول مرة أخرى قريبًا.")
            break
        response = chat_with_gpt(user_input)
        print(f"GPT-2: {response}\n")


AttributeError: module 'torch.backends' has no attribute 'xformers'

In [ ]:
# pip install accelerate
import torch
from transformers import pipeline

pipe = pipeline(model="openai-community/gpt2", torch_dtype=torch.bfloat16, device_map="auto")
output = pipe("This is a cool example!", do_sample=True, top_p=0.95)
print(output)

In [2]:
print(output)

[{'generated_text': "This is a cool example!\n\nWhat I'm really interested in here is what's currently going on. For us at Microsoft, the problem of a company that depends on hardware for its business is really one of performance and the question is how do"}]


In [10]:
!pip install torchvision # First install torchvision if it is not already installed
!pip install --upgrade torch torchvision torchaudio # Update all PyTorch libraries together to ensure compatibility

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.5 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
  Attempting uninstall: torch
    Found existing installation: torch 2.6.0
    Uninstalling torch-2.6.0:
      Successfully uninstalled torch-2.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xformers 0.0.30.dev974 requires torch==2.6.0, but you have torch 2.5.1 which is incompatible.


  Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached triton-3.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.4 kB)
Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl (766.7 MB)
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 179, in resolve
    self.factory.preparer

In [ ]:
!pip3 install torch torchvision torchaudio

In [4]:
# pip install accelerate
import torch
from transformers import pipeline
#torch.backends.xformers.enabled = True
pipe = pipeline(model="openai-community/gpt2", torch_dtype=torch.bfloat16, device_map="auto")
pipe.enable_xformers_memory_efficient_attention()
output = pipe("This is a cool example!", do_sample=True, top_p=0.95)
print(output)

Device set to use cuda:0


AttributeError: 'TextGenerationPipeline' object has no attribute 'enable_xformers_memory_efficient_attention'

In [1]:
!pip install xformers

  Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached triton-3.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.4 kB)
Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl (766.7 MB)
Using cached triton-3.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (253.2 MB)
  Attempting uninstall: triton
    Found existing installation: triton 3.1.0
    Uninstalling triton-3.1.0:
      Successfully uninstalled triton-3.1.0
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1
    Uninstalling torch-2.5.1:
      Successfully uninstalled torch-2.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.20.1+cu124 requires torch==2.5.1, but you have torch 2.6.0 which is incompatible.
torchaudio 2.5.1+cu124 requires torch==2.5.1, but you have torch 2.6.0 which is 

In [3]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 51.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1


In [1]:
import xformers

In [2]:
!pip install torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0

  Using cached torchvision-0.21.0-cp311-cp311-manylinux1_x86_64.whl.metadata (6.1 kB)
  Using cached torchaudio-2.6.0-cp311-cp311-manylinux1_x86_64.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 91.7 MB/s eta 0:00:00
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.20.1+cu124
    Uninstalling torchvision-0.20.1+cu124:
      Successfully uninstalled torchvision-0.20.1+cu124
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.5.1+cu124
    Uninstalling torchaudio-2.5.1+cu124:
      Successfully uninstalled torchaudio-2.5.1+cu124
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.18 requires torch<2.6,>=1.10, but you have torch 2.6.0 which is incompatible.


In [2]:
import torch
from transformers import pipeline
import xformers  # Import xformers

# Enable xformers memory efficient attention
pipe = pipeline(model="openai-community/gpt2", torch_dtype=torch.bfloat16, device_map="auto")

# Check if xformers is available and enable it
if hasattr(pipe.model, 'enable_xformers_memory_efficient_attention'):
    pipe.model.enable_xformers_memory_efficient_attention()
else:
    print("xformers is not available for this model.")

output = pipe("This is a cool example!", do_sample=True, top_p=0.95)
print(output)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


xformers is not available for this model.
[{'generated_text': "This is a cool example!\n\nBut don't do it, because it will render the HTML into JavaScript and slow your code down\n\nLet's add a custom helper function which will take an array and convert it into a string\n\nvar"}]


شغال

In [ ]:
!pip install torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0

import torch
from transformers import pipeline
import xformers  # Import xformers

# Enable xformers memory efficient attention
pipe = pipeline(model="openai-community/gpt2", torch_dtype=torch.bfloat16, device_map="auto")

# Check if xformers is available and enable it
if hasattr(pipe.model, 'enable_xformers_memory_efficient_attention'):
    pipe.model.enable_xformers_memory_efficient_attention()
else:
    print("xformers is not available for this model.")

output = pipe("This is a cool example!", do_sample=True, top_p=0.95)
print(output)

In [3]:


import torch
from transformers import pipeline
import xformers
pipe = pipeline(model="openai-community/gpt2", torch_dtype=torch.bfloat16, device_map="auto", enable_xformers_memory_efficient_attention=True)
#pipe.model.enable_xformers_memory_efficient_attention()
output = pipe("This is a cool example!", do_sample=True, top_p=0.95)
print(output)

Device set to use cuda:0


ValueError: The following `model_kwargs` are not used by the model: ['enable_xformers_memory_efficient_attention'] (note: typos in the generate arguments will also show up in this list)

In [ ]:
!pip install transformers accelerate bitsandbytes
!pip install --upgrade transformers

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# تحميل النموذج والتوكنيزر
model_id = "EleutherAI/gpt-neo-125M" # يمكنك استبداله بنموذج آخر
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_8bit=True, # لتحميل النموذج بتنسيق 8 بت لتوفير مساحة الذاكرة
)

# تفعيل xformers
if hasattr(model, "enable_xformers_memory_efficient_attention"):
    model.enable_xformers_memory_efficient_attention()
else:
    print("xformers is not available for this model.")

# إنشاء pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    torch_dtype=torch.float16,
)

# توليد النص
prompt = "اكتب قصة قصيرة عن قطة فضائية:"
generated_text = pipe(prompt, max_new_tokens=100, do_sample=True, top_p=0.95)[0]['generated_text']

# طباعة النص المولد
print(generated_text)

In [4]:
from transformers import pipeline

pipe = pipeline("text-generation", model="EleutherAI/gpt-neo-125M")
pipe.enable_xformers_memory_efficient_attention() # قد يؤدي هذا إلى خطأ إذا لم يكن مدعومًا

# ... بقية الكود ...

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

Device set to use cuda:0


AttributeError: 'TextGenerationPipeline' object has no attribute 'enable_xformers_memory_efficient_attention'

شغالXFormers

In [1]:
import torch
from transformers import pipeline
import xformers  # Import xformers

# Enable xformers memory efficient attention
pipe = pipeline(model="EleutherAI/gpt-neo-125M", torch_dtype=torch.bfloat16, device_map="auto")

# Check if xformers is available and enable it
if hasattr(pipe.model, 'enable_xformers_memory_efficient_attention'):
    pipe.model.enable_xformers_memory_efficient_attention()
else:
    print("xformers is not available for this model.")

output = pipe("This is a cool example!", do_sample=True, top_p=0.95)
print(output)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


xformers is not available for this model.
[{'generated_text': 'This is a cool example!\nIt would be nice to see a lot of the stuff you did already!\n\nI wish'}]


18ث

شغال

In [1]:
import torch
from transformers import pipeline
#import xformers  # Import xformers

# Enable xformers memory efficient attention
pipe = pipeline(model="EleutherAI/gpt-neo-125M", torch_dtype=torch.bfloat16, device_map="auto")
"""
# Check if xformers is available and enable it
if hasattr(pipe.model, 'enable_xformers_memory_efficient_attention'):
    pipe.model.enable_xformers_memory_efficient_attention()
else:
    print("xformers is not available for this model.")
"""

output = pipe("This is a cool example!", do_sample=True, top_p=0.95)
print(output)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'This is a cool example! Here’s the short video you should watch. It’s not that many pictures,'}]


20ث

In [1]:
from transformers import pipeline
generator = pipeline('text-generation', model='EleutherAI/gpt-neo-125M')
generator("EleutherAI has", do_sample=True, min_length=20)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'EleutherAI has discovered some hidden secrets about the real history of the Christian faith. When a Christian family member comes to'}]

شغال

In [1]:
from transformers import pipeline
generator = pipeline('text-generation', model='EleutherAI/gpt-neo-125M')
generator("Who is Napoleon Bonaparte?", do_sample=True, min_length=50)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1427: UserWarning: Unfeasible length constraints: `min_length` (50) is larger than the maximum possible length (27). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length. Note that `max_length` is set to 27, its default value.
  warnings.w

[{'generated_text': 'Who is Napoleon Bonaparte?\n\nA small nation, divided along racial, ideological differences, and having more than 100 million Americans'}]

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# تفعيل خيار XFormers للذاكرة الفعالة لعمليات الانتباه
import torch
if torch.cuda.is_available():
    torch.backends.xformers.enabled = True
    print("XFormers is enabled.")

# تحميل النموذج والتوكنайزر
model_name = "openai-community/gpt2"  # يمكنك استبداله بنموذجًا آخر مثل "gpt2-medium", "gpt2-large"، إلخ
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

def chat_with_gpt(prompt, max_length=50):
    # ترميز النص المدخل
    inputs = tokenizer(prompt, return_tensors="pt")
    # إنتاج الرد من النموذج
    outputs = model.generate(**inputs, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2, temperature=0.7)
    # فك ترميز الرد إلى نص سهل القراءة
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

if __name__ == "__main__":
    print("مرحبا بك في شاشة الشات مع نموذج GPT-2!")
    while True:
        user_input = input("أنت: ")
        if user_input.lower() in ["خروج", "exit", "quit"]:
            print("شكراً! حاول مرة أخرى قريبًا.")
            break
        response = chat_with_gpt(user_input)
        print(f"GPT-2: {response}\n")


AttributeError: module 'torch.backends' has no attribute 'xformers'

شغال شات ديالوجXFormers

In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Check if CUDA is available and if xformers is installed correctly
if torch.cuda.is_available():
    try:
        import xformers
        torch.backends.xformers.enabled = True  # Try to enable xformers
        print("XFormers is enabled.")
    except (ImportError, AttributeError):
        print("XFormers is not available or not enabled. Please check your installation and compatibility.")

# تحميل النموذج والتوكنайزر
model_name = "openai-community/gpt2"  # يمكنك استبداله بنموذجًا آخر مثل "gpt2-medium", "gpt2-large"، إلخ
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

def chat_with_gpt(prompt, max_length=50):
    # ترميز النص المدخل
    inputs = tokenizer(prompt, return_tensors="pt")
    # إنتاج الرد من النموذج
    outputs = model.generate(**inputs, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2, temperature=0.7)
    # فك ترميز الرد إلى نص سهل القراءة
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

if __name__ == "__main__":
    print("مرحبا بك في شاشة الشات مع نموذج GPT-2!")
    while True:
        user_input = input("أنت: ")
        if user_input.lower() in ["خروج", "exit", "quit"]:
            print("شكراً! حاول مرة أخرى قريبًا.")
            break
        response = chat_with_gpt(user_input)
        print(f"GPT-2: {response}\n")

XFormers is not available or not enabled. Please check your installation and compatibility.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


مرحبا بك في شاشة الشات مع نموذج GPT-2!
أنت: hi


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


GPT-2: hi, who was born in the city of Kolkata, was a member of the Kargil movement. He was also a student of Srinagar University.

"I was in a state of shock when I heard about the incident



KeyboardInterrupt: Interrupted by user

شغالxformers

In [1]:
import torch
from transformers import pipeline
import xformers  # Import xformers

# Enable xformers memory efficient attention
pipe = pipeline(model="EleutherAI/gpt-neo-125M", torch_dtype=torch.bfloat16, device_map="auto")

# Check if xformers is available and enable it
if hasattr(pipe.model, 'enable_xformers_memory_efficient_attention'):
    pipe.model.enable_xformers_memory_efficient_attention()
else:
    print("xformers is not available for this model.")

output = pipe("This is a cool example!", do_sample=True, top_p=0.95)
print(output)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


xformers is not available for this model.
[{'generated_text': 'This is a cool example! We use it for two reasons - One is to get a sense of how our code can work with'}]


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Check if CUDA is available and if xformers is installed correctly
if torch.cuda.is_available():
    try:
        import xformers
        torch.backends.xformers.enabled = True  # Try to enable xformers
        print("XFormers is enabled.")
    except (ImportError, AttributeError):
        print("XFormers is not available or not enabled. Please check your installation and compatibility.")

# تحميل النموذج والتوكنайزر
model_name = "openai-community/gpt2"  # يمكنك استبداله بنموذجًا آخر مثل "gpt2-medium", "gpt2-large"، إلخ
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

def chat_with_gpt(prompt, max_length=50):
    # ترميز النص المدخل
    inputs = tokenizer(prompt, return_tensors="pt")
    # إنتاج الرد من النموذج
    outputs = model.generate(**inputs, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2, temperature=0.7)
    # فك ترميز الرد إلى نص سهل القراءة
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

if __name__ == "__main__":
    print("مرحبا بك في شاشة الشات مع نموذج GPT-2!")
    while True:
        user_input = input("أنت: ")
        if user_input.lower() in ["خروج", "exit", "quit"]:
            print("شكراً! حاول مرة أخرى قريبًا.")
            break
        response = chat_with_gpt(user_input)
        print(f"GPT-2: {response}\n")

https://huggingface.co/blog/sd_distillation

In [ ]:

from diffusers import DiffusionPipeline
import torch

pipeline = DiffusionPipeline.from_pretrained("segmind/small-sd", torch_dtype=torch.float16)
prompt = "Portrait of a pretty girl"
negative_prompt = "(deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck"
image = pipeline(prompt, negative_prompt = negative_prompt).images[0]
image.save("my_image.png")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

tokenizer/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer/vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

...

diffusion_pytorch_model.bin:   0%|          | 0.00/335M [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--segmind--small-sd/snapshots/f8b087e68071a29bbe85a4b07e6e1e8998d59aa7/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--segmind--small-sd/snapshots/f8b087e68071a29bbe85a4b07e6e1e8998d59aa7/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--segmind--small-sd/snapshots/f8b087e68071a29bbe85a4b07e6e1e8998d59aa7/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--segmind--small-sd/snapshots/f8b087e68071a29bbe85a4b07e6e1e8998d59aa7/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


  0%|          | 0/50 [00:00<?, ?it/s]

https://huggingface.co/blog/sd_distillation

In [ ]:

from diffusers import DiffusionPipeline
import torch

pipeline = DiffusionPipeline.from_pretrained("segmind/small-sd", torch_dtype=torch.float16)
prompt = "Portrait of a pretty girl"
negative_prompt = "(deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck"
image = pipeline(prompt, negative_prompt = negative_prompt).images[0]
image.save("my_image.png")


شغالجيدا

https://github.com/segmind/distill-sd/blob/master/inference.py

In [1]:

from diffusers import DiffusionPipeline
import torch

pipeline = DiffusionPipeline.from_pretrained("segmind/small-sd", torch_dtype=torch.float16, device_map="balanced")
pipeline.enable_vae_slicing()
pipeline.enable_vae_tiling()
pipeline.enable_xformers_memory_efficient_attention()
prompt = "Portrait of a pretty girl"

#negative_prompt = "(deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck"
image = pipeline(prompt).images[0]
image.save("my_image.png")
#37ث

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--segmind--small-sd/snapshots/f8b087e68071a29bbe85a4b07e6e1e8998d59aa7/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--segmind--small-sd/snapshots/f8b087e68071a29bbe85a4b07e6e1e8998d59aa7/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--segmind--small-sd/snapshots/f8b087e68071a29bbe85a4b07e6e1e8998d59aa7/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--segmind--small-sd/snapshots/f8b087e68071a29bbe85a4b07e6e1e8998d59aa7/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:

from diffusers import DiffusionPipeline
import torch

pipeline = DiffusionPipeline.from_pretrained("segmind/small-sd", torch_dtype=torch.float16, device_map="balanced")
pipeline.enable_vae_slicing()
pipeline.enable_vae_tiling()
pipeline.enable_xformers_memory_efficient_attention()
prompt = "Portrait of a pretty girl"

#negative_prompt = "(deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck"
image = pipeline(prompt).images[0]
image.save("my_image.png")
#37ث

In [2]:
import torch
from diffusers import DiffusionPipeline
from diffusers import DPMSolverMultistepScheduler
from torch import Generator


path = 'segmind/small-sd' #model card
prompt = "Faceshot Portrait of pretty young (18-year-old) Caucasian wearing a high neck sweater, (masterpiece, extremely detailed skin, photorealistic, heavy shadow, dramatic and cinematic lighting, key light, fill light), sharp focus, BREAK epicrealism"
negative_prompt = "(deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck"

torch.set_grad_enabled(False)
torch.backends.cudnn.benchmark = True

with torch.inference_mode():
    gen = Generator("cuda")
    gen.manual_seed(1674753452)
    pipe = DiffusionPipeline.from_pretrained(path, torch_dtype=torch.float16, safety_checker=None, requires_safety_checker=False)
    pipe.to('cuda')
    pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
    pipe.unet.to(device='cuda', dtype=torch.float16, memory_format=torch.channels_last)

    for i in range(3):
        img = pipe(prompt=prompt,negative_prompt=negative_prompt, width=512, height=512, num_inference_steps=25, guidance_scale = 7, num_images_per_prompt=1, generator = gen).images[0]
        img.save(f"{i}.png")

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--segmind--small-sd/snapshots/f8b087e68071a29bbe85a4b07e6e1e8998d59aa7/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--segmind--small-sd/snapshots/f8b087e68071a29bbe85a4b07e6e1e8998d59aa7/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--segmind--small-sd/snapshots/f8b087e68071a29bbe85a4b07e6e1e8998d59aa7/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--segmind--small-sd/snapshots/f8b087e68071a29bbe85a4b07e6e1e8998d59aa7/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


ValueError: `final_sigmas_type` zero is not supported for `algorithm_type` deis. Please choose `sigma_min` instead.

شغال

In [1]:
import torch
from diffusers import DiffusionPipeline
from diffusers import DPMSolverMultistepScheduler
from torch import Generator


path = 'segmind/small-sd' #model card
prompt = "Faceshot Portrait of pretty young (18-year-old) Caucasian wearing a high neck sweater, (masterpiece, extremely detailed skin, photorealistic, heavy shadow, dramatic and cinematic lighting, key light, fill light), sharp focus, BREAK epicrealism"
negative_prompt = "(deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck"

torch.set_grad_enabled(False)
torch.backends.cudnn.benchmark = True

with torch.inference_mode():
    gen = Generator("cuda")
    gen.manual_seed(1674753452)
    pipe = DiffusionPipeline.from_pretrained(path, torch_dtype=torch.float16, safety_checker=None, requires_safety_checker=False)
    pipe.to('cuda')
    # Get the scheduler configuration and update 'final_sigmas_type'
    scheduler_config = pipe.scheduler.config
    scheduler_config['final_sigmas_type'] = 'sigma_min'  # Change to 'sigma_min'

    # Create the scheduler with the updated configuration
    pipe.scheduler = DPMSolverMultistepScheduler.from_config(scheduler_config)
    pipe.unet.to(device='cuda', dtype=torch.float16, memory_format=torch.channels_last)

    for i in range(3):
        img = pipe(prompt=prompt,negative_prompt=negative_prompt, width=512, height=512, num_inference_steps=25, guidance_scale = 7, num_images_per_prompt=1, generator = gen).images[0]
        img.save(f"{i}.png")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--segmind--small-sd/snapshots/f8b087e68071a29bbe85a4b07e6e1e8998d59aa7/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--segmind--small-sd/snapshots/f8b087e68071a29bbe85a4b07e6e1e8998d59aa7/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--segmind--small-sd/snapshots/f8b087e68071a29bbe85a4b07e6e1e8998d59aa7/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--segmind--small-sd/snapshots/f8b087e68071a29bbe85a4b07e6e1e8998d59aa7/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

شرح الأوامر باللغة العربية:
1. torch.set_grad_enabled(False):

هذا الأمر يقوم بتعطيل حساب التدرجات (gradients) في PyTorch.

التدرجات هي قيم تُستخدم في تدريب النماذج العصبية لتحديث معلمات النموذج وتحسين أدائه. عند القيام بـ الاستدلال (inference)، أي استخدام النموذج المُدرّب بالفعل لتوليد مخرجات جديدة، لا نحتاج إلى حساب التدرجات.

لذلك، يتم استخدام torch.set_grad_enabled(False) لتسريع عملية الاستدلال وتوفير مساحة الذاكرة، حيث لا يتم تخزين التدرجات.

2. torch.backends.cudnn.benchmark = True:

هذا الأمر يقوم بتفعيل خيار التحسين التلقائي (auto-tuner) في مكتبة cuDNN، وهي مكتبة تُستخدم لتسريع عمليات التعلم العميق على أجهزة GPU من NVIDIA.

التحسين التلقائي يعني أن cuDNN ستبحث عن أفضل خوارزمية لتنفيذ العمليات على GPU الخاص بك، مما يؤدي إلى تحسين الأداء وتسريع عملية الاستدلال.

لكن، هذا التحسين التلقائي قد يستغرق بعض الوقت في البداية، لذلك يُفضّل استخدامه فقط عند تشغيل النموذج عدة مرات بنفس حجم المدخلات.

بإختصار:

torch.set_grad_enabled(False): لتعطيل حساب التدرجات وتسريع الاستدلال.
torch.backends.cudnn.benchmark = True : لتحسين أداء cuDNN على GPU وتسريع الاستدلال.
أتمنى أن يكون هذا الشرح واضحًا ومفيدًا!

شغال

In [1]:
import torch
from diffusers import DiffusionPipeline
from diffusers import DPMSolverMultistepScheduler
from torch import Generator


path = 'segmind/small-sd' #model card
prompt = "Faceshot Portrait of pretty young (18-year-old) Caucasian wearing a high neck sweater, (masterpiece, extremely detailed skin, photorealistic, heavy shadow, dramatic and cinematic lighting, key light, fill light), sharp focus, BREAK epicrealism"
negative_prompt = "(deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck"

torch.set_grad_enabled(False)
torch.backends.cudnn.benchmark = True

with torch.inference_mode():
    gen = Generator("cuda")
    gen.manual_seed(1674753452)
    pipe = DiffusionPipeline.from_pretrained(path, torch_dtype=torch.float16, safety_checker=None, requires_safety_checker=False)
    pipe.to('cuda')
    # Get the scheduler configuration and update 'final_sigmas_type'
    scheduler_config = pipe.scheduler.config
    scheduler_config['final_sigmas_type'] = 'sigma_min'  # Change to 'sigma_min'

    # Create the scheduler with the updated configuration
    pipe.scheduler = DPMSolverMultistepScheduler.from_config(scheduler_config)
    pipe.unet.to(device='cuda', dtype=torch.float16, memory_format=torch.channels_last)

    for i in range(1):
        img = pipe(prompt=prompt,negative_prompt=negative_prompt, width=1024, height=1024, num_inference_steps=50, guidance_scale = 7, num_images_per_prompt=1, generator = gen).images[0]
        img.save(f"{i}.png")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--segmind--small-sd/snapshots/f8b087e68071a29bbe85a4b07e6e1e8998d59aa7/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--segmind--small-sd/snapshots/f8b087e68071a29bbe85a4b07e6e1e8998d59aa7/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--segmind--small-sd/snapshots/f8b087e68071a29bbe85a4b07e6e1e8998d59aa7/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--segmind--small-sd/snapshots/f8b087e68071a29bbe85a4b07e6e1e8998d59aa7/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
path = 'segmind/small-sd' #model card
prompt = "Beautiful 16 year old naked teen with slim body and amazing toes"
negative_prompt = "(deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck"


In [3]:
import torch
from diffusers import DiffusionPipeline
from diffusers import DPMSolverMultistepScheduler
from torch import Generator


path = 'segmind/small-sd' #model card
prompt = "dog"
negative_prompt = "(deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck"

torch.set_grad_enabled(False)
torch.backends.cudnn.benchmark = True

with torch.inference_mode():
    gen = Generator("cuda")
    gen.manual_seed(1673452)
    pipe = DiffusionPipeline.from_pretrained(path, torch_dtype=torch.float16, safety_checker=None, requires_safety_checker=False)
    pipe.to('cuda')
    # Get the scheduler configuration and update 'final_sigmas_type'
    scheduler_config = pipe.scheduler.config
    scheduler_config['final_sigmas_type'] = 'sigma_min'  # Change to 'sigma_min'

    # Create the scheduler with the updated configuration
    pipe.scheduler = DPMSolverMultistepScheduler.from_config(scheduler_config)
    pipe.unet.to(device='cuda', dtype=torch.float16, memory_format=torch.channels_last)

    for i in range(1):
        img = pipe(prompt=prompt,negative_prompt=negative_prompt, width=1024, height=1024, num_inference_steps=50, guidance_scale = 7, num_images_per_prompt=1, generator = gen).images[0]
        img.save(f"{i}.png")

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--segmind--small-sd/snapshots/f8b087e68071a29bbe85a4b07e6e1e8998d59aa7/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--segmind--small-sd/snapshots/f8b087e68071a29bbe85a4b07e6e1e8998d59aa7/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--segmind--small-sd/snapshots/f8b087e68071a29bbe85a4b07e6e1e8998d59aa7/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--segmind--small-sd/snapshots/f8b087e68071a29bbe85a4b07e6e1e8998d59aa7/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


  0%|          | 0/50 [00:00<?, ?it/s]

https://github.com/segmind/distill-sd/blob/master/inference.py

نموذج مقطر

 50ث بدون اكسفورمرز ولكن استخدم تسريعات تورش
 شغال

In [1]:
import torch
from diffusers import DiffusionPipeline
from diffusers import DPMSolverMultistepScheduler
from torch import Generator


path = 'segmind/small-sd' #model card
prompt = "dog"
negative_prompt = "(cgi, 3d, render, sketch, cartoon, drawing  anime, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck"

torch.set_grad_enabled(False)
torch.backends.cudnn.benchmark = True

with torch.inference_mode():
    gen = Generator("cuda")
    gen.manual_seed(1673452)
    pipe = DiffusionPipeline.from_pretrained(path, torch_dtype=torch.float16, safety_checker=None, requires_safety_checker=False)
    pipe.to('cuda')
    # Get the scheduler configuration and update 'final_sigmas_type'
    scheduler_config = pipe.scheduler.config
    scheduler_config['final_sigmas_type'] = 'sigma_min'  # Change to 'sigma_min'

    # Create the scheduler with the updated configuration
    pipe.scheduler = DPMSolverMultistepScheduler.from_config(scheduler_config)
    pipe.unet.to(device='cuda', dtype=torch.float16, memory_format=torch.channels_last)

    for i in range(1):
        img = pipe(prompt=prompt,negative_prompt=negative_prompt, width=1024, height=1024, num_inference_steps=50, guidance_scale = 7, num_images_per_prompt=1, generator = gen).images[0]
        img.save(f"{i}.png")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--segmind--small-sd/snapshots/f8b087e68071a29bbe85a4b07e6e1e8998d59aa7/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--segmind--small-sd/snapshots/f8b087e68071a29bbe85a4b07e6e1e8998d59aa7/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--segmind--small-sd/snapshots/f8b087e68071a29bbe85a4b07e6e1e8998d59aa7/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--segmind--small-sd/snapshots/f8b087e68071a29bbe85a4b07e6e1e8998d59aa7/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
import torch
from diffusers import DiffusionPipeline
from diffusers import DPMSolverMultistepScheduler
from torch import Generator


path = 'segmind/small-sd' #model card
prompt = "Faceshot Portrait of pretty young (18-year-old) Caucasian wearing a high neck sweater, (masterpiece, extremely detailed skin, photorealistic, heavy shadow, dramatic and cinematic lighting, key light, fill light), sharp focus, BREAK epicrealism"
negative_prompt = "(deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck"

torch.set_grad_enabled(False)
torch.backends.cudnn.benchmark = True

with torch.inference_mode():
    gen = Generator("cuda")
    gen.manual_seed(1674753452)
    pipe = DiffusionPipeline.from_pretrained(path, torch_dtype=torch.float16, safety_checker=None, requires_safety_checker=False)
    pipe.to('cuda')
    # Get the scheduler configuration and update 'final_sigmas_type'
    scheduler_config = pipe.scheduler.config
    scheduler_config['final_sigmas_type'] = 'sigma_min'  # Change to 'sigma_min'

    # Create the scheduler with the updated configuration
    pipe.scheduler = DPMSolverMultistepScheduler.from_config(scheduler_config)
    pipe.unet.to(device='cuda', dtype=torch.float16, memory_format=torch.channels_last)

    for i in range(1):
        img = pipe(prompt=prompt,negative_prompt=negative_prompt, width=1024, height=1024, num_inference_steps=50, guidance_scale = 7, num_images_per_prompt=1, generator = gen).images[0]
        img.save(f"{i}.png")